<a href="https://colab.research.google.com/github/tayfununal/continuous_CNN/blob/main/resnet50_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf 
import numpy as np 

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# expand new axis, channel axis 
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# [optional]: we may need 3 channel (instead of 1)
x_train = np.repeat(x_train, 3, axis=-1)
x_test = np.repeat(x_test, 3, axis=-1)

# it's always better to normalize 
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# resize the input shape , i.e. old shape: 28, new shape: 32
x_train = tf.image.resize(x_train, [32,32]) # if we want to resize 
x_test = tf.image.resize(x_test, [32,32]) # if we want to resize 

# one hot 
y_train = tf.keras.utils.to_categorical(y_train , num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test , num_classes=10)

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [ ]:
np.random.seed(8)
tf.random.set_seed(8)

input = tf.keras.Input(shape=(32,32,3))
efnet = tf.keras.applications.ResNet50(weights='imagenet',
                                             include_top = False, 
                                             input_tensor = input)
# Now that we apply global max pooling.
gap = tf.keras.layers.GlobalMaxPooling2D()(efnet.output)

# Finally, we add a classification layer.
output = tf.keras.layers.Dense(10, activation='softmax', use_bias=True)(gap)

# bind all
func_model = tf.keras.Model(efnet.input, output)

In [ ]:
func_model.compile(
          loss  = tf.keras.losses.CategoricalCrossentropy(),
          metrics = tf.keras.metrics.CategoricalAccuracy(),
          optimizer = tf.keras.optimizers.Adam())
# fit 
func_model.fit(x_train, y_train, batch_size=128, epochs=5, verbose = 2)

In [ ]:
predictions = func_model.predict(x_test)
print(predictions[0])

In [ ]:
func_model.evaluate(x_test,  y_test, batch_size=10000, verbose=1)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(4, 12, figsize=(30, 14))  # 4 = row , 12= column , 30 = horizontal size , 14 = vertical size

for i in range(0,4):
  for j in range(0,12):
    random_int = np.random.randint(1,2000,1)
    ax[i][j].imshow(x_test[int(random_int)], cmap=plt.cm.binary)
    ax[i][j].set_title("Value : {} \nPredicted Value : {}".format(int(np.where(y_test[random_int])[1]),np.argmax(predictions[random_int])))

plt.show()